In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot

In [2]:
STATUS={
            "PLAF" : "Plan order, created automatically after the order has been created", 
			"ERO" : "Not Planned, not started. May have missing parts", 
            "FREI" : "No missing part, production order free", 
            "TRÜC": "1 level of a 2 level produciton has been finished", 
            "RÜCK" : "Production finished, on the way to CSC", 
            "GLFT" : "Delivered to CSC",
            "LOE" : "P Order Deleted/Finished",
            "TGLI" : "Partical delivery, the production order has been splitted and partical delivered",
            "" : "UNKNOWN"
    }

In [18]:
def fileRead(filename) :
    items=[] #all materials in the doc
    item=[] #single material in parsing
    count=0 #to count the number of "-" so we could identify the boundary. 

    with open(filename,"r", encoding="ISO-8859-1") as file: #encoding based on the sap output

        #To skip the headers accounting 6 six lines. 
        for i in range(6) :
            line=file.readline() 


        #read a single line for getting so & po info from header
        #Order type      TD       Document No.  2018630587 P.O. number   4601958823           Vendor number    
        line=file.readline()     
        so=line.split()[5]
        po=line.split()[8]

        while line.find('*****')<0: # when meeting *****  E N D   ******, loop ends 
            line=file.readline()
            if line.find("Seite") >0 :line=file.readline();continue #Seite should be skipped

            #info started with "----------------", so it counts its number 
            if line[0]=='-':  count +=1                       

            # conditions for adding line to item[]
            if line[0]=='|' or line.count('Ident-code') :
                item.append(line)

            if (line[0]=='-' and item!=[] and count%2==1) :            
                items.append(item)
                item=[]            
    items.append(item)  
    return items, so, po

In [19]:
def itemsParsing(items, so, po) :
    
    TotalList=[]
    SumList=[]

    #For each items 
    start=1
    if str(items).find("Ship-to party ")>0 :
        start=2        
    
    for row in range(start,len(items)) :
        stored=None
        n=len(items[row])
        
        if str(items[row]).find("stored in")>0 : 
            n=n-1
            stored=items[row][-2][76:91].strip()
        
        status=items[row][0][91:95].strip()
        #print(items[row])
        
        #so=items[row][2][26:36]
        no=int(items[row][0][1:5].strip())
        partno=items[row][0][14:22].strip()
        config=items[row][-1][36:80].strip() if items[row][-1][36:80].count("-")>3 else items[1][0][23:62].strip()

        #Each records of each Items Parsing
        T1=0;T2=0;T3=0;T4=0
        if str(items[row]).find("Ident")>0 : n-=1
        
        soidx=items[row][0][1:5]
        poidx=items[row][1][1:5]
        d2=None
        d3=None
        d4=None
        eta=None
        for i in range(2,n) :
            t=items[row][i]

            if t[11:19].find(".")>0 :
                d1=t[11:19]
                #d1=datetime.datetime.strptime(t[11:19],  "%y.%m.%d").date()
            
            if t[35:46].find(".")>0 :
                d2=t[38:46]
                #d2=datetime.datetime.strptime(t[38:46],  "%y.%m.%d").date() 
                eta=d2

            if t[85:93].find(".")>0 and t[85:93].find("00.00.00")==-1: 
                d3=t[85:93]
                #d3=datetime.datetime.strptime(t[85:93],  "%y.%m.%d").date()
            #else : 
                #d3=''
            
            if t[116:124].find(".")>0 : 
                d4=t[116:124]
                #d4=datetime.datetime.strptime(t[116:124],  "%y.%m.%d").date()
            #else : 
                #d4=''
				
            t1=0;t2=0;t3=0;t4=0
            if t[22:25].strip()!='' : 
                t1=int(t[22:25])
                T1+=int(t1)    
            
            if t[49:52].strip()!='' : 
                t2=int(t[49:52]) 
                T2+=int(t2)    

            if t[96:99].strip() !="" and t[96:99].find("ED")==-1 : 
                t3=int(t[96:99]);
                T3+=t3
                
            if t[127:130].strip() !="" and t[127:130].find("|")==-1 : 
                t4=int(t[127:130]);
                T4+=t4

            #Invoice#     
            reference=t[-31:-23].strip()

            #A single row created here
            TotalList.append([so, soidx.strip(), po, poidx.strip(), (so+ str(no)).strip(), d1,t1,d2,t2,d3,t3,d4,t4,reference])

        delta=T1-T4
        reference=""
        if delta==0:
            etd=TotalList[-1][-3]
            REMARK="Invoiced"
            reference=items[row][-2][-31:-23]
        else :
            REMARK=STATUS[status]
            etd=None

        partial="Partial" if delta>0 and T1>delta else ''
        SumList.append([so, soidx.strip(), po, poidx.strip(), (so+ str(no)).strip(), partno, config, T1,T2,T3,T4, delta, stored, etd, eta, status,REMARK + " " + reference, reference, partial])

    return TotalList, SumList

In [22]:
def detailDF (conDF, list) :
    
    DF=pd.DataFrame([], columns=["so", "soidx", "po", "poidx" "d1", "t1", "d2", "t2", "d3","t3", "d4", "t4", "reference"])

    for i in list:
        print ("DETAIL DF", i)
        DF=pd.concat([DF, pd.DataFrame(conDF.loc[i], columns=["d1", "t1", "d2", "t2", "d3","t3", "d4", "t4", "reference"])])
        #print(pd.DataFrame(conDF.loc[str(i)], columns=["d1", "t1", "d2", "t2", "d3","t3", "d4", "t4", "reference"]))
		#partialDF=pd.concat([partialDF, pd.DataFrame(conDF.loc[i])])
    return DF

In [23]:
def mergeDF (filelist) :
    colDF=["so", "soidx", "po", "poidx", "idx", "d1", "t1", "d2", "t2", "d3","t3", "d4", "t4", "reference"]
    colSUM=["so", "soidx", "po","poidx", "idx", "partno", "config","T1","T2","T3","T4","△","Stored", "ETD", "ETA", "Status", "Remark", "Ref#", "Partial"]
    conDF=pd.DataFrame([], columns=colDF)
    conSUM=pd.DataFrame([], columns=colSUM)

    for file in filelist :
        print (file, "read to be parsed.... ")
        items, so, po =fileRead(file)        
        TotalList, SumList = itemsParsing(items, so, po)
        #print("============")
        #print (file, TotalList)
        #print("============")
        conDF=pd.concat([conDF,pd.DataFrame(TotalList, columns=colDF)])
        conSUM=pd.concat([conSUM,pd.DataFrame(SumList, columns=colSUM)])
        
    
    conDF=pd.DataFrame(conDF, columns=colDF)                         
    conSUM=pd.DataFrame(conSUM, columns=colSUM)
    conDF=conDF.set_index("idx")
    conSUM=conSUM.set_index("idx") 
    partialDF = detailDF(conDF, conSUM[conSUM.Partial=='Partial'].index)    
    return conDF, conSUM, partialDF

## Test for a single file

colDF=["so", "soidx", "po", "poidx", "idx", "d1", "t1", "d2", "t2", "d3","t3", "d4", "t4", "reference"]
colSUM=["so", "soidx", "po","poidx", "idx", "partno", "config","T1","T2","T3","T4","△","Stored", "ETD", "ETA", "Status", "Remark", "Ref#", "Partial"]
conDF=pd.DataFrame([], columns=colDF)
conSUM=pd.DataFrame([], columns=colSUM)

items,so, po = fileRead("2018630587_test")
TotalList, SumList=itemsParsing(items,so,po)
conDF=pd.concat([conDF,pd.DataFrame(TotalList, columns=colDF)])
conSUM=pd.concat([conSUM,pd.DataFrame(SumList, columns=colSUM)])

conDF=pd.DataFrame(conDF, columns=colDF)                         
conSUM=pd.DataFrame(conSUM, columns=colSUM)
conDF=conDF.set_index("idx")
conSUM=conSUM.set_index("idx") 
partialDF = detailDF(conDF, conSUM[conSUM.Partial=='Partial'].index)    

list1=(list(conSUM.iloc[0]))
yva1data = str(list1).replace("[","").replace("]","").replace("None","Null")

# Running for all files in the list

In [24]:
filelist=[]
f=open("YVA1list.txt","r", encoding="UTF-8")
for line in f :
    filelist.append(line.splitlines()[0])

DF,SUM,PARTIAL = mergeDF(filelist)



2018683983 read to be parsed.... 
2018692836 read to be parsed.... 
2018715879 read to be parsed.... 
2018715876 read to be parsed.... 
2018740360 read to be parsed.... 
2018740334 read to be parsed.... 
2018740335 read to be parsed.... 
2018740337 read to be parsed.... 
2018759852 read to be parsed.... 


In [25]:
SUM

,so,soidx,po,poidx,partno,config,T1,T2,T3,T4,△,Stored,ETD,ETA,Status,Remark,Ref#,Partial
idx,,,,,,,,,,,,,,,,,,
201868398310,2018683983,10,4601974229,1,8024917,ELGA-TB-KF-150-3750-0H-ZL,1,1,1,1,0,None,18.05.17,18.05.17,GLFT,Invoiced 20946828,20946828,
201868398320,2018683983,20,4601974229,2,8024917,ELGA-TB-KF-150-1990-0H,1,1,0,0,1,None,None,18.06.01,ERO,"Not Planned, not started. May have missing parts",,
201868398330,2018683983,30,4601974229,3,8024916,ELGA-TB-KF-120-2500-0H-ZL,2,2,0,0,2,None,None,18.06.15,ERO,"Not Planned, not started. May have missing parts",,
201868398340,2018683983,40,4601974229,4,8024916,ELGA-TB-KF-120-1350-0H-ZL,1,1,0,0,1,None,None,18.06.15,ERO,"Not Planned, not started. May have missing parts",,
201868398350,2018683983,50,4601974229,5,8024916,ELGA-TB-KF-120-600-0H,1,1,0,0,1,None,None,18.06.15,ERO,"Not Planned, not started. May have missing parts",,
201868398360,2018683983,60,4601974229,6,556825,EGC-HD-220-600-TB-0H-GP,1,1,0,0,1,None,None,18.07.02,ERO,"Not Planned, not started. May have missing parts",,
201868398370,2018683983,70,4601974229,7,556825,EGC-HD-220-1600-TB-0H-GP,1,1,0,0,1,None,None,18.07.02,ERO,"Not Planned, not started. May have missing parts",,
201868398380,2018683983,80,4601974229,8,556825,EGC-HD-220-2035-TB-0H-GP-KL,1,1,0,0,1,None,None,18.07.02,ERO,"Not Planned, not started. May have missing parts",,
201868398390,2018683983,90,4601974229,9,556825,EGC-HD-220-940-TB-0H-GP,1,1,0,0,1,None,None,18.07.02,ERO,"Not Planned, not started. May have missing parts",,


In [ ]:
#To save excel file 
writer=pd.ExcelWriter('output.xlsx')
SUM.to_excel(writer,'SUM')
DF.to_excel(writer,'DF')
PARTIAL.to_excel(writer,'PARTIAL')
writer.save()

## Input into MYSQL DB(schema = LO)

In [36]:
#!/usr/bin/python
import time
## dd/mm/yyyy format
print (time.strftime("%Y%m%d"))

20180521


In [26]:
import mysql.connector
f=open('mysql.key').readline().split()
cnx = mysql.connector.connect(user=f[0], password=f[1],
                              host='127.0.0.1',
                              database='lo')
cursor = cnx.cursor()      

## Create table

In [ ]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cnx.database = 'lo'  
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

### Delete the previous YVA (to be developed to keep the history)

In [ ]:
query = "delete from yva1"
cursor.execute(query)
cnx.commit()

## Create Table

In [ ]:
TABLES={}
TABLES['yva1'] = (
    "CREATE TABLE yva1 ("
    "  so varchar(12) NOT NULL,"
    "  soidx varchar(4) NOT NULL,"
    "  po varchar(10),"
    "  poidx varchar(4),"
    "  partno varchar(10),"    
    "  config varchar(50),"    
    "  T1 int(4),"    
    "  T2 int(4),"    
    "  T3 int(4),"    
    "  T4 int(4),"  
    "  delta int(4),"  
    "  stock varchar(30),"  
    "  etd date,"
    "  eta date,"
    "  status varchar(10),"
    "  remark1 varchar(50),"
    "  remark2 varchar(50),"
    "  remark3 varchar(50),"    
    "  PRIMARY KEY (so,soidx)"
    ") ENGINE=InnoDB")

In [ ]:
cursor.execute(TABLES['yva1'])

### insert the new YVA1 data to the DB

In [30]:
cnx = mysql.connector.connect(user=f[0], password=f[1],
                              host='127.0.0.1',
                              database='lo')

cursor = cnx.cursor()    
for k,v in SUM.iterrows():
    value= (str(list(v)).replace("[","").replace("]","").replace("None","Null"))
    query = ("INSERT INTO yva1 VALUES(" + value + ") ")
    print(query)
    
    try :
        cursor.execute(query)
    except Exception as e:
        print(e)
        
cnx.commit()
cnx.close()

INSERT INTO yva1 VALUES('2018683983', '10', '4601974229', '1', '8024917', 'ELGA-TB-KF-150-3750-0H-ZL', 1, 1, 1, 1, 0, Null, '18.05.17', '18.05.17', 'GLFT', 'Invoiced 20946828', '20946828', '') 
1062 (23000): Duplicate entry '2018683983-10' for key 'PRIMARY'
INSERT INTO yva1 VALUES('2018683983', '20', '4601974229', '2', '8024917', 'ELGA-TB-KF-150-1990-0H', 1, 1, 0, 0, 1, Null, Null, '18.06.01', 'ERO', 'Not Planned, not started. May have missing parts ', '', '') 
1062 (23000): Duplicate entry '2018683983-20' for key 'PRIMARY'
INSERT INTO yva1 VALUES('2018683983', '30', '4601974229', '3', '8024916', 'ELGA-TB-KF-120-2500-0H-ZL', 2, 2, 0, 0, 2, Null, Null, '18.06.15', 'ERO', 'Not Planned, not started. May have missing parts ', '', '') 
1062 (23000): Duplicate entry '2018683983-30' for key 'PRIMARY'
INSERT INTO yva1 VALUES('2018683983', '40', '4601974229', '4', '8024916', 'ELGA-TB-KF-120-1350-0H-ZL', 1, 1, 0, 0, 1, Null, Null, '18.06.15', 'ERO', 'Not Planned, not started. May have missing pa

In [49]:
from os import listdir
from os.path import isfile, join
mypath="."
#onlyfiles = [f for f in listdir(mypath) if (isfile(join(mypath, f) and f.find("2018") == 0))]
fielist = [f for f in listdir(mypath) if f.find("2018") == 0]